In [ ]:
pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.multitest import fdrcorrection

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

matplotlib.rcParams['pdf.fonttype'] = 42

import os
import dxpy

In [ ]:
# Input and output files
ICD_CHAPTER_DIR="/path/to/ICD/Chapter/data" # Use the output directory of script 3_Data preparation/UKB/5_parse_ICD10.ipynb
BURDEN_DIR="/path/to/UKB/burden/tables" # Use the output directory of script 3_Data_preparation/UKB/3_identify_samples.ipynb
OUTPUT_DIR="/path/to/output/directory"
# Output will be (1) a CSV file with regression statistics as presented in Table S5I
# and (2) a PDF showing ICD chapters associated with 16p12.1 deletion as presented in Fig S5A

In [ ]:
# Load ICD10 Chapters
df=pd.read_csv(f'{ICD_CHAPTER_DIR}/ICD10_16p12_Chapter.csv')
df['Case_Control']='16p12.1 deletion'
df2=pd.read_csv(f'{ICD_CHAPTER_DIR}/ICD10_control_Chapter.csv')
df2['Case_Control']='Control'
df=pd.concat([df, df2])

In [ ]:
# Load sex and YOB information
pdf=pd.read_csv(f'{BURDEN_DIR}/16p_burden.csv')
pdf2=pd.read_csv(f'{BURDEN_DIR}/control_burden.csv')
pdf=pd.concat([pdf, pdf2])
pdf=pdf[['Sample', 'YOB', 'Sex']]

In [ ]:
df=pd.merge(df, pdf, on='Sample', how='left')

In [ ]:
# Identify significant associations of the 16p12.1 deletion with ICD10 chapters using logistic regression
phenos=['Neoplasms', 'Blood', 'Endocrine/Metabolic', 'Mental/behavioral disorders', 'Nervous system',
        'Eye', 'Ear', 'Circulatory system', 'Respiratory system', 'Digestive system', 'Skin/subcutaeous tissue',
        'Musc. system/connective tissue', 'Genitourinary system', 'Pregnancy/childbirth', 'Congenital malformations']
# Normalize YOB
df['YOB']=(df['YOB']-df['YOB'].mean(skipna=True))/df['YOB'].std(skipna=True)
# Binarize Case_Control status
df['Case_Control']=df.Case_Control.map({'16p12.1 deletion':1, 'Control':0})

In [ ]:
def run_model(moddf, input_vars, output_col):
    X=sm.add_constant(moddf[input_vars].to_numpy())
    mod=sm.Logit(moddf[output_col].to_numpy(), X)
    
    res=mod.fit(maxiter=5000)

    # Parse model
    num_vars=len(input_vars)+1
    ci=res.conf_int(alpha=0.05)
    r2=res.prsquared

    res_dict={'Phenotype':[output_col]*num_vars, 'Variable':['Intercept']+input_vars, 'Test':['Logistic regression']*num_vars, 'N':[moddf.shape[0]]*num_vars,
              'Estimate':res.params, 'Error':res.bse, '95% C.I. lower':[i[0] for i in ci], '95% C.I. upper':[i[1] for i in ci], 'p value':res.pvalues, 'R2':[r2]*num_vars}
    mod_res=pd.DataFrame(res_dict)
    
    return mod_res

In [ ]:
outdf=pd.DataFrame(columns=['Phenotype', 'Variable', 'Test', 'Estimate', 'Error', '95% C.I. lower', '95% C.I. upper', 'p value', 'R2'])
for pheno in phenos:
    print(pheno)
    # Get model inputs
    input_cols=['Sex', 'YOB', 'Case_Control']
    moddf=df[~df[input_cols+[pheno]].isnull().any(axis=1)].copy()

    # Remove sex as input for female-specific phenotypes
    if pheno=='Pregnancy/childbirth':
        input_cols=['YOB', 'Case_Control']

    # Scale age
    moddf['YOB']=(moddf['YOB']-moddf['YOB'].mean())/moddf['YOB'].std()
    
    out=run_model(moddf, input_cols, pheno)
    outdf=pd.concat([outdf, out])

In [ ]:
# FDR correction
outdf['BH FDR']=np.nan
outdf.loc[outdf.Variable=='Case_Control', 'BH FDR']=fdrcorrection(outdf[outdf.Variable=='Case_Control']['p value'].to_numpy())[1]

In [ ]:
# Save results
outdf.to_csv('ICD_regressions.csv', index=False)
dxpy.upload_local_file('ICD_regressions.csv', folder=OUTPUT_DIR, parents=True)
os.remove('ICD_regressions.csv')

In [ ]:
# Make a volcano plot of significant results
outdf=outdf[outdf.Variable=='Case_Control']
outdf['log10p']=-np.log10(outdf['p value'])

sns.scatterplot(outdf[outdf['BH FDR']>=0.05], x='Estimate', y='log10p', hue='Phenotype', hue_order=phenos, alpha=1, palette=sns.color_palette("Spectral", n_colors=15), legend=False)
sns.scatterplot(outdf[outdf['BH FDR']<=0.05], x='Estimate', y='log10p', hue='Phenotype', hue_order=phenos, alpha=1, palette=sns.color_palette("Spectral",  n_colors=15), legend=False, edgecolor='k')

# Add lines and term names
for idx, row in outdf.iterrows():
    # Add labels if significant
    if row['BH FDR']<=0.05:
        plt.text(row['Estimate'], row['log10p']+0.1, row['Phenotype'], ha='center', fontsize=5)
plt.tight_layout()
plt.savefig('ICD10_enrichment.pdf')
plt.close()

In [ ]:
# Save plot
dxpy.upload_local_file('ICD10_enrichment.pdf', folder=OUTPUT_DIR, parents=True)
os.remove('ICD10_enrichment.pdf')